In [1]:
import sys
import os
sys.path.append(os.getcwd())
import pandas as pd
import string 
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer as stemmer 
import nltk
from nltk.corpus import stopwords
from collections import Counter
import nltk.stem

from functools import reduce
import collections 


# Special vocabulary module from shoyu
import vocabulary_hdp as vocab

In [2]:
stemmer = stemmer("english")

In [3]:
def preprocess(doc):
    '''Function that lemmatizes words in abstract by verbs'''
    
    return [stemmer.stem(WordNetLemmatizer().lemmatize(w, pos='v')) for w in doc.translate(str.maketrans('','', string.punctuation)).lower().split(' ')]

def rm_stopwords_and_short_words(words, st_words):
    '''Function removes stop words and those with length < 3'''
    results = []
    for i in words:
        if not i in st_words and len(i)  > 3:
            results.append(i)
    return results

def full_preprocess(doc, st_words):
    '''Performs word lemmatization and stopword removal'''
    return rm_stopwords_and_short_words(preprocess(doc), st_words)


In [4]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ecoronado/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ecoronado/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df = pd.read_csv("tm_test_data.csv")

In [5]:
st_words = stopwords.words('english') # made this an argument of the functions

In [6]:
# place docs into dicitonary structure to be used in helper function below

in_docs = {k: str(txt[0]) for k,txt in enumerate(df.values)}

In [7]:
# Helper function
def tf(docs, st_words):
    '''Function that calculates the term frequencies of word from an text-document paired dictionary input. 
       The output is a term frequency table '''
    
    # generate counts per document
    counts = {k: Counter(full_preprocess(txt, st_words)) for k, txt in docs.items()}
    
    tf_df = pd.DataFrame.from_dict(counts).fillna(0).astype(int) # build pandas df, fill empty vals with 0s
    
    return(tf_df)


In [8]:
# created term frequency df with pre-processing functions
tf_df = tf(in_docs, st_words)

In [9]:
# Filter out tokens that appear in fewer than 3 abstracts and tokens that appear in more than half the abstracts 

filtered_df = tf_df[(tf_df.sum(axis=1) > 3)]
filtered_df = filtered_df[(filtered_df.astype(bool).sum(axis=1) / tf_df.shape[1] < 0.5)]

In [10]:
# quickly build a dictionary based on filtered dataframe

filt_words = pd.DataFrame.to_dict(filtered_df)

In [11]:
# gets non-zero words per document and creates nested list structure

tokenized_df_ab  = [[word for word, cnt in words.items() if cnt!=0] for dkeys, words in filt_words.items()]

In [12]:
voca = vocab.Vocabulary()
docs = [voca.doc_to_ids(doc) for doc in tokenized_df_ab]
beta = .5

In [19]:
tokenized_df_ab[0]

['paper',
 'high',
 'interact',
 'userfriend',
 'lisp',
 'program',
 'introduc',
 'perform',
 'homogen',
 'analysi',
 'brief',
 'introduct',
 'techniqu',
 'present',
 'well',
 'modif',
 'presenc',
 'miss',
 'algorithm',
 'discuss',
 'overview',
 'object',
 'orient',
 'code',
 'produc',
 'dialog',
 'plot',
 'order',
 'demonstr',
 'main',
 'featur',
 'small',
 'larg',
 'dataset',
 'analyz',
 'final',
 'comparison',
 'make',
 'current',
 'avail']

In [37]:
#tokenized_df = [full_preprocess(i,st_words) for i in df.abstract]

In [38]:
#all_words = reduce(lambda x,y: x+y, tokenized_df )
#all_words =  list(set(all_words))

In [39]:

# Filter out tokens that appear in fewer than 3 abstracts and tokens that appear in more than half the abstracts 
#all_words_counts = np.zeros(len(all_words))

#for k,i in enumerate(all_words):
#    for j in tokenized_df:
#        if i in j:
#            all_words_counts[k] += 1 
            
#word_counts_dict = list(zip(all_words, list(all_words_counts)))
#word_counts_dict_ab = list(filter(lambda x: x[1] > 3, word_counts_dict))
#word_counts_dict_ab2 = list(filter(lambda x: x[1] < len(tokenized_df)/2, word_counts_dict_ab))
#final_dict = [i[0] for i in word_counts_dict_ab2]

In [40]:
#tokenized_df_ab = []
#for i in tokenized_df:
#    tokenized_df_ab.append([j for j in i if j in final_dict])
    

In [41]:
#voca = vocab.Vocabulary()
#docs = [voca.doc_to_ids(doc) for doc in tokenized_df_ab]
#beta = .5

In [61]:
! jupyter nbconvert --to script 'data_prep.ipynb'

[NbConvertApp] Converting notebook data_prep.ipynb to script
[NbConvertApp] Writing 2210 bytes to data_prep.py
